In [ ]:
import pandas as pd
import geopandas as gpd
from os import path, environ, makedirs
from dotenv import load_dotenv
from unidecode import unidecode

from core.geo import areal_weighted_interpolation

In [ ]:
load_dotenv()

# Carregando os dados extraídos no notebook anterior

Neste notebook, vamos utilizar os dados extraídos e salvos pelo notebook `03 habitação - extração.ipynb`.

In [ ]:
input_dir = path.join('data', 'input', 'urbanismo')

In [ ]:
filename = path.join(input_dir, 'orcamento_habitacao_original.csv')
df_orcamento = pd.read_csv(filename,
            sep=';',
            decimal=',',
            encoding='utf8',
            dtype=str)
df_orcamento

In [ ]:
for col in [col for col in df_orcamento.columns if 'Vl' in col]:
    df_orcamento[col] = df_orcamento[col].astype(float)
df_orcamento['DataExtracao'] = pd.to_datetime(df_orcamento['DataExtracao'])
df_orcamento

In [ ]:
filename = path.join(input_dir, 'orcamento_regionalizado_habitacao_original.csv')
df_orcamento_r = pd.read_csv(filename,
            sep=';',
            decimal=',',
            encoding='utf8',
            dtype=str)
df_orcamento_r

In [ ]:
df_orcamento_r['VALOR_DETALHAMENTO_AÇÃO'] = df_orcamento_r['VALOR_DETALHAMENTO_AÇÃO'].astype(float)
df_orcamento_r['DATA_EXTRAÇÃO'] = pd.to_datetime(df_orcamento_r['DATA_EXTRAÇÃO'])
df_orcamento_r

In [ ]:
df_orcamento_r.dtypes

In [ ]:
filename = path.join(input_dir, 'pdm_meta_12_original.csv')
df_meta_12 = pd.read_csv(filename,
            sep=';',
            decimal=',',
            encoding='utf8')
df_meta_12

In [ ]:
filename = path.join(input_dir, 'his_entregue_original.csv')
df_his = pd.read_csv(filename,
            sep=';',
            decimal=',',
            encoding='utf8')
df_his

In [ ]:
filename = path.join(input_dir, 'tpu_emitido_original.csv')
df_tpu = pd.read_csv(filename,
            sep=';',
            decimal=',',
            encoding='utf8')
df_tpu

In [ ]:
filename = path.join(input_dir, 'favelas_original.gpkg')
df_favelas = gpd.read_file(filename)
df_favelas

Como vamos precisar fazer a interseção espacial entre os dados de favelas e o mapa de subprefeituras, vamos carregar também o mapa de subprefeituras, que foi salvo pelo notebook `01 areas de risco - extração.ipynb`.

In [ ]:
filename = path.join(input_dir, 'subprefeituras_original.gpkg')
gdf_subs = gpd.read_file(filename)
gdf_subs

# Transformação e padronização

Nos indicadores de habitação, apenas o ano e as subprefeituras são presentes em vários arquivos. Mas, além disso, os arquivos precisarão de tratamentos específicos. Vamos começar com os mais simples, onde é necessário apenas padronizar as subprefeituras.

Primeiro, vamos carregar os dados de subprefeituras que serão utilizados no Qlik Sense.

## CSV de Subprefeituras do Qlik

In [ ]:
url_subs = environ.get('CSV_SUBPREFEITURAS_QLIK')
df_subs = pd.read_csv(url_subs)
df_subs

In [ ]:
df_subs = df_subs[['sub.CODIGO', 'sub.NOME']]
df_subs

## Chave composta subprefeitura-ano

Como 3 tabelas possuem valores para mais de um ano, também vale a pena a criação de uma chave composta entre subprefeitura e ano. A tabela que possui mais períodos é a tabela da meta 12 do Programa de Metas, com os anos de 2021, 2022, 2023 e 2024. Vamos criar uma tabela com o produto cartesiano entre subprefeituras e anos.

In [ ]:
df_subs_ano = (
    df_subs[['sub.NOME']]
    .merge(pd.Series(data=[2021, 2022, 2023, 2024], name='ano'),
           how='cross')
)

df_subs_ano.loc[:, 'subprefeitura-ano'] = (
    df_subs_ano.loc[:, 'sub.NOME'] + ' | ' + df_subs_ano.loc[:, 'ano'].astype(str)
)

df_subs_ano

## PdM - Meta 12: Prover 49.000 moradias de interesse social

In [ ]:
df_meta_12

In [ ]:
df_meta_12 = df_meta_12.loc[:, ['Subprefeitura', '2021', '2022', '2023', '2024']]

df_meta_12

In [ ]:
df_meta_12 = df_meta_12.melt('Subprefeitura',
                var_name='ano',
                value_name='qtd_unidades_acumulado')

df_meta_12

Como os valores foram divulgados no acumulado entre 2021 e 2024, vamos calcular o incremento de cada ano antes de carregar os dados no Qlik, mas mantendo as duas colunas.

In [ ]:
df_meta_12['qtd_unidades'] = df_meta_12.groupby('Subprefeitura')['qtd_unidades_acumulado'].diff()
df_meta_12

In [ ]:
df_meta_12.loc[df_meta_12['ano']=='2021', 'qtd_unidades'] = (
    df_meta_12.loc[df_meta_12['ano']=='2021', 'qtd_unidades_acumulado'])
df_meta_12

Finalmente, vamos criar uma coluna com os nomes padronizados de subprefeituras.

In [ ]:
subs_meta_12 = df_meta_12['Subprefeitura'].apply(unidecode).unique().tolist()
subs_meta_12.sort()
subs_meta_12

In [ ]:
subs_qlik = df_subs['sub.NOME'].unique().tolist()
subs_qlik.sort()
subs_qlik

In [ ]:
len(subs_meta_12)

Vemos que existem 3 subprefeituras faltantes no dataframe da meta 12. Vamos avaliar quais podem ser. Numa inspeção detalhada vemos que faltam `ARICANDUVA-FORMOSA-CARRAO`, `SAO MIGUEL` e `VILA MARIANA`. Vamos criar uma cópia da lista de subs do qlik adaptada à meta 12.

In [ ]:
subs_qlik_meta_12 = subs_qlik.copy()
subs_qlik_meta_12.remove('ARICANDUVA-FORMOSA-CARRAO')
subs_qlik_meta_12.remove('SAO MIGUEL')
subs_qlik_meta_12.remove('VILA MARIANA')
subs_qlik_meta_12

In [ ]:
mapper_meta_12 = {
    o: q
    for o, q in zip(subs_meta_12, subs_qlik_meta_12)
}

mapper_meta_12

In [ ]:
df_meta_12.insert(1,
                  'sub.NOME',
                  df_meta_12['Subprefeitura'].apply(unidecode).map(mapper_meta_12))
df_meta_12

In [ ]:
df_meta_12['qtd_unidades'] = df_meta_12['qtd_unidades'].astype(int)
df_meta_12['qtd_unidades_acumulado'] = df_meta_12['qtd_unidades_acumulado'].astype(int)
df_meta_12['ano'] = df_meta_12['ano'].astype(int)
df_meta_12

In [ ]:
df_meta_12 = df_meta_12.merge(df_subs_ano,
                              how='left',
                              on=['sub.NOME', 'ano'])

df_meta_12

## Produção de habitação de interesse social

In [ ]:
df_his

In [ ]:
subs_his = df_his['região'].apply(unidecode).unique().tolist()
subs_his.sort()
subs_his

In [ ]:
mapper_his = {
    s: q for s, q in zip(subs_his, subs_qlik)
}

mapper_his

In [ ]:
df_his.insert(1,
                  'sub.NOME',
                  df_his['região'].apply(unidecode).map(mapper_his))
df_his

In [ ]:
df_his['qtd_unidades'] = df_his['qtd_unidades'].astype(int)
df_his['ano'] = df_his['ano'].astype(int)
df_his

In [ ]:
df_his = df_his.merge(df_subs_ano,
                              how='left',
                              on=['sub.NOME', 'ano'])

df_his

## Número de termos de Permissão de Uso (TPU) emitidos em nome da mulher da familia

In [ ]:
df_tpu['qtd_termos'] = df_tpu['qtd_termos'].astype(int)
df_tpu['ano'] = df_tpu['ano'].astype(int)
df_tpu

## Número de domicílios em favelas

Como os domicílios em favelas não possuem uma coluna identificando a subprefeitura, precisamos fazer a interseção espacial entre os dados de favelas e o mapa de subprefeituras.

Primeiro, vamos chegar se existem favelas que estão dentro de mais de uma subprefeitura.

In [ ]:
df_fav_sub = df_favelas.overlay(gdf_subs, how='intersection')
df_fav_sub.iloc[:,0].describe()

Existem 8 registros duplicados, o que indica que algumas favelas estão em mais de uma subprefeitura. Portanto, um spatial join simples não funcionará. Vamos fazer a interseção espacial entre os dois dataframes.

### Interpolação ponderada por área

In [ ]:
df_fav_sub = areal_weighted_interpolation(
    left=df_favelas,
    right=gdf_subs,
    right_id_col='nm_subprefeitura',
    original_var_name='Particular permanente',
    final_var_name='domicilios_particulares_permanentes_favela'
)
df_fav_sub

In [ ]:
df_favelas['Particular permanente'].sum()

In [ ]:
df_fav_sub['domicilios_particulares_permanentes_favela'].sum() 

In [ ]:
df_favelas['Particular permanente'].sum()-df_fav_sub['domicilios_particulares_permanentes_favela'].sum()

In [ ]:
1-df_fav_sub['domicilios_particulares_permanentes_favela'].sum()/df_favelas['Particular permanente'].sum()

Cerca de 659 domicílios em favelas não foram alocados a nenhuma subprefeitura. Isso representa apenas 0,1% do total de domicílios em favelas, então não é uma perda grave. Porém, o caso mais provável é de que parte dessas favelas estejam fora do limite das subprefeituras, o que não deveria ocorrer com os dados de favelas filtrados apenas para o município de São Paulo. Vamos investigar.

### Avaliando domicílios não alocados

In [ ]:
df_favelas_fora = df_favelas.overlay(gdf_subs, how='difference', keep_geom_type=True)
df_favelas_fora

Como era esperado, existem favelas que estão parcialmente fora do limite do município de São Paulo. Vamos inspecionar visualmente onde estão essas favelas.

In [ ]:
m = gdf_subs.explore(
    tiles='CartoDB positron',
    tooltip=True,
    popup=True,
    style_kwds=dict(color='grey', fill=False))

m = df_favelas_fora.explore(
    m=m,
    legend=True,
    tooltip=True,
    popup=True)

m

De fato, existem favelas que estão parcialmente fora do limite do município de São Paulo, e são onde existem as áreas mais significativas de favelas fora das subprefeituras. Porém, também existem casos áreas de favelas que estão completamente dentro do município de São Paulo, mas que não foram alocadas a nenhuma subprefeitura por se situarem exatamente na divisa entre subprefeituras, mas provavelmente isso não representa uma área grande o suficiente para distorcer as estimativas.

### Identificando áreas fora dos limites das subprefeituras

De todo modo, vamos calcular a proporção de domicílios dessas áreas que não foram alocados a nenhuma subprefeitura, e aplicar essa proporção para redistribuir os domicílios não alocados entre as subprefeituras. Com a proporção de domicílios calculada, podemos atribuir os domicílios não alocados à subprefeitura mais próxima de cada área de favela não alocada.

In [ ]:
df_favelas_fora['area_total'] = df_favelas_fora.apply(
    lambda row: df_favelas.query(f'cd_fcu=="{row.cd_fcu}"').geometry.area.iloc[0],
    axis=1)

df_favelas_fora

In [ ]:
df_favelas_fora['area_fora'] = df_favelas_fora.geometry.area
df_favelas_fora

In [ ]:
df_favelas_fora['peso'] = (
    df_favelas_fora['area_fora'] / df_favelas_fora['area_total']
)
df_favelas_fora

In [ ]:
df_favelas_fora['domicilios_favela_fora'] = (
    df_favelas_fora['peso'] * df_favelas_fora['Particular permanente']
).round().astype(int)
df_favelas_fora

In [ ]:
df_favelas_fora['domicilios_favela_fora'].sum()

De fato, 655 dos 659 domicílios não alocados estão em favelas que estão parcialmente fora do limite do município de São Paulo. Os outros 4 domicílios provavelmente são falhas de arredondamento. Antes de atribuir os domicílios não alocados, às subprefeituras mais próximas, vamos excluir do dataframe as áreas sem domicílios.

In [ ]:
df_favelas_fora = df_favelas_fora[df_favelas_fora['domicilios_favela_fora']>0]
df_favelas_fora

### Atribuindo domicílios não alocados às subprefeituras mais próximas

In [ ]:
df_favelas_fora = df_favelas_fora.sjoin_nearest(
    gdf_subs[['nm_subprefeitura', 'geometry']],
    how='left'
).drop(columns='index_right')

df_favelas_fora

Como algumas áreas de favelas estão em mais de uma subprefeitura, precisamos remover uma das duplicatas. Vamos manter a primeira ocorrência.

In [ ]:
df_favelas_fora = df_favelas_fora.drop_duplicates(subset=['cd_fcu']).reset_index(drop=True)

df_favelas_fora

Agora, podemos agregar o número de domicílios em favelas por subprefeitura e adicionar ao dataframe anterior.

In [ ]:
df_fav_sub_fora = (
    df_favelas_fora
    .groupby('nm_subprefeitura', as_index=False)
    .agg(
        domicilios_favela_fora = ('domicilios_favela_fora', 'sum')
    )
)

df_fav_sub_fora

### Agregando os domicílios em favelas por subprefeitura

In [ ]:
subs_fora = df_fav_sub['nm_subprefeitura'].isin(df_fav_sub_fora['nm_subprefeitura'])

df_fav_sub.loc[subs_fora, 'domicilios_particulares_permanentes_favela'] = (
    df_fav_sub
    .loc[subs_fora]
    .apply(lambda row: row['domicilios_particulares_permanentes_favela']
           + df_fav_sub_fora.loc[df_fav_sub_fora['nm_subprefeitura']==row['nm_subprefeitura'], 'domicilios_favela_fora'].values[0], axis=1)
)

df_fav_sub

In [ ]:
df_fav_sub['domicilios_particulares_permanentes_favela'].sum()

In [ ]:
df_favelas['Particular permanente'].sum()

In [ ]:
df_favelas['Particular permanente'].sum()-df_fav_sub['domicilios_particulares_permanentes_favela'].sum()

In [ ]:
1-df_fav_sub['domicilios_particulares_permanentes_favela'].sum()/df_favelas['Particular permanente'].sum()

Agora sim, temos uma perda de apenas 4 domicílios em favelas, dentro do universo de 649.765 domicílios em favelas, o que representa uma perda de apenas 0,0006%. Ademais, essa perda provavelmente se deve a falhas de arredondamento, então podemos considerar que a estimativa está satisfatória.

### Padronizando os nomes de subprefeituras

In [ ]:
subs_favelas = df_fav_sub['nm_subprefeitura'].apply(unidecode).unique().tolist()
subs_favelas.sort()
subs_favelas

In [ ]:
len(subs_favelas)

O tamanho da lista de subs indica que uma das subprefeituras não possui nenhum domicílio em favelas. Inspecionando a lista, vemos que a subprefeitura faltante é a de Pinheiros, o que parece fazer sentido. Primeiro, vamos criar uma cópia da lista de subs do qlik adaptada aos dados de domicílios em favelas.

In [ ]:
subs_qlik_favelas = subs_qlik.copy()
subs_qlik_favelas.remove('PINHEIROS')
subs_qlik_favelas

In [ ]:
mapper_favelas = {
    s: q for s, q in zip(subs_favelas, subs_qlik_favelas)
}

mapper_favelas

In [ ]:
any(s != q for s, q in mapper_favelas.items())

### Recriando com base no df_subs

In [ ]:
df_fav_sub_final = (
    df_subs
    .merge(df_fav_sub[['nm_subprefeitura',
                          'sg_subprefeitura',
                          'domicilios_particulares_permanentes_favela']],
                how='left',
                left_on='sub.NOME',
                right_on='nm_subprefeitura')
    .drop(columns=['nm_subprefeitura'])
)

df_fav_sub_final


Finalmente, completamos os campos vazios da subprefeitura de Pinheiros.

In [ ]:
df_fav_sub_final['domicilios_particulares_permanentes_favela'] = (
    df_fav_sub_final['domicilios_particulares_permanentes_favela']
    .fillna(0)
    .astype(int)
)

df_fav_sub_final.loc[df_fav_sub_final['sub.NOME']=='PINHEIROS', 'sg_subprefeitura'] = (
    gdf_subs.loc[gdf_subs['nm_subprefeitura']=='PINHEIROS', 'sg_subprefeitura'].values[0]
)

df_fav_sub_final

## Orçamento do Programa de habitação

Para o orçamento, além de padronizar os nomes de subprefeituras e tipos de dados das métricas, precisaremos também adaptar os dados para compatibilizar o orçamento regionalizado e não realizado. Para isso, vamos fazer o seguinte:

1. Classificar o orçamento detalhado por nível de regionalização nas seguintes categorias: subprefeitura, região e não regionalizável;
1. Agrupar o restante do orçamento não detalhado e manter apenas o orçamento inicial, atualizado e liquidado;
1. Subtrair o total do orçamento detalhado do orçamento não detalhado e classificar o nível de regionalização como não regionalizado;
1. Unir os dois dataframes de orçamento de acordo com as dimensões mantidas.

### Orçamento regionalizado

In [ ]:
df_orcamento_r.head(1)

In [ ]:
cols_orcamento_r = ['CÓDIGO_ÓRGÃO', 'SIGLA_ÓRGÃO', 'DESCRIÇÃO_ÓRGÃO',
                    'CÓDIGO_PROGRAMA', 'DESCRIÇÃO_PROGRAMA',
                    'CÓDIGO_PROJ_ATIV', 'DESCRIÇÃO_PROJ_ATIV',
                    'CÓDIGO_VÍNCULO_PMSP', 'REGIÃO',
                    'SUBPREFEITURA', 'TIPO_REGIONALIZAÇÃO']

cols_orcamento_r_vl = ['VALOR_DETALHAMENTO_AÇÃO']

df_orcamento_r = df_orcamento_r[cols_orcamento_r + cols_orcamento_r_vl]
df_orcamento_r

In [ ]:
df_orcamento_r['TIPO_REGIONALIZAÇÃO'].value_counts()

In [ ]:
df_orcamento_r.loc[df_orcamento_r['TIPO_REGIONALIZAÇÃO'].isna(), 'TIPO_REGIONALIZAÇÃO'] = 'Despesa Não-Regionalizável'
df_orcamento_r

In [ ]:
df_orcamento_r['TIPO_REGIONALIZAÇÃO'].value_counts()

In [ ]:
df_orcamento_r = df_orcamento_r.groupby(cols_orcamento_r).sum().round(2).reset_index()

df_orcamento_r

In [ ]:
df_orcamento_r.loc[
    ~df_orcamento_r['REGIÃO'].str.contains('Supra', na=False),
    'NIVEL_REGIONALIZAÇÃO'] = 'Região'

df_orcamento_r

In [ ]:
df_orcamento_r.loc[
    ~df_orcamento_r['SUBPREFEITURA'].str.contains('Supra', na=False),
    'NIVEL_REGIONALIZAÇÃO'] = 'Subprefeitura'

df_orcamento_r

In [ ]:
df_orcamento_r.loc[df_orcamento_r['NIVEL_REGIONALIZAÇÃO'].isna(), 'NIVEL_REGIONALIZAÇÃO'] = 'Não regionalizável'
df_orcamento_r

### Orçamento não regionalizado

In [ ]:
df_orcamento.head(1)

In [ ]:
cols_orcamento = ['Cd_Orgao', 'Sigla_Orgao', 'Ds_Orgao', 'Cd_Programa',
                  'Ds_Programa', 'ProjetoAtividade', 'Ds_Projeto_Atividade',
                  'COD_VINC_REC_PMSP']

cols_orcamento_vl = ['Vl_Orcado_Ano', 'Vl_Orcado_Atualizado', 'Vl_Liquidado']

df_orcamento_original = df_orcamento.copy()
df_orcamento = df_orcamento[cols_orcamento + cols_orcamento_vl]
df_orcamento

In [ ]:
df_orcamento = df_orcamento.groupby(cols_orcamento).sum().reset_index()
df_orcamento

In [ ]:
r_agg_cols = ['CÓDIGO_ÓRGÃO', 'CÓDIGO_PROGRAMA', 'CÓDIGO_PROJ_ATIV',
              'CÓDIGO_VÍNCULO_PMSP']

agg_cols = ['Cd_Orgao', 'Cd_Programa', 'ProjetoAtividade',
            'COD_VINC_REC_PMSP']

df_orcamento_r_agg = (
    df_orcamento_r[r_agg_cols + ['VALOR_DETALHAMENTO_AÇÃO']]
    .groupby(r_agg_cols)
    .sum()
    .reset_index()
)

df_orcamento_r_agg.loc[:, 'VALOR_DETALHAMENTO_AÇÃO'] = (
    df_orcamento_r_agg
    .loc[:, 'VALOR_DETALHAMENTO_AÇÃO']
    .round(2)
)

df_orcamento_ajustado = df_orcamento.merge(
    df_orcamento_r_agg,
    left_on=agg_cols,
    right_on=r_agg_cols,
    how='left'
).drop(columns=r_agg_cols)

df_orcamento_ajustado

In [ ]:
df_orcamento_ajustado.loc[df_orcamento_ajustado['VALOR_DETALHAMENTO_AÇÃO'].isna(), 'VALOR_DETALHAMENTO_AÇÃO'] = 0

df_orcamento_ajustado.loc[:, 'Vl_Liquidado_N_Detalhado'] = (
    df_orcamento_ajustado.loc[:, 'Vl_Liquidado']
    - df_orcamento_ajustado.loc[:, 'VALOR_DETALHAMENTO_AÇÃO']).round(2)

df_orcamento_ajustado

In [ ]:
df_orcamento_ajustado[df_orcamento_ajustado['Vl_Liquidado_N_Detalhado']<0]

In [ ]:
df_orcamento_ajustado[['Vl_Liquidado', 'VALOR_DETALHAMENTO_AÇÃO']].sum()

### Unindo os dados de orçamento

Agora, vamos adicionar os dados não detalhados ao dataframe que contém o orçamento detalhado.

In [ ]:
orcamento_cols_map = {'Cd_Orgao': 'CÓDIGO_ÓRGÃO',
                      'Sigla_Orgao': 'SIGLA_ÓRGÃO',
                      'Ds_Orgao': 'DESCRIÇÃO_ÓRGÃO',
                      'Cd_Programa': 'CÓDIGO_PROGRAMA',
                      'Ds_Programa': 'DESCRIÇÃO_PROGRAMA',
                      'ProjetoAtividade': 'CÓDIGO_PROJ_ATIV',
                      'Ds_Projeto_Atividade': 'DESCRIÇÃO_PROJ_ATIV',
                      'COD_VINC_REC_PMSP': 'CÓDIGO_VÍNCULO_PMSP',
                      'Vl_Liquidado_N_Detalhado': 'Vl_Liquidado'}

df_orcamento_ajustado = (
    df_orcamento_ajustado
    .drop(columns=['VALOR_DETALHAMENTO_AÇÃO', 'Vl_Liquidado'])
    .rename(columns=orcamento_cols_map)
    )

df_orcamento_ajustado

In [ ]:
df_orcamento_r = (df_orcamento_r
                  .rename(columns={'VALOR_DETALHAMENTO_AÇÃO': 'Vl_Liquidado'}))

df_orcamento_r

In [ ]:
df_orcamento_final = pd.concat([df_orcamento_r, df_orcamento_ajustado])

df_orcamento_final

In [ ]:
df_orcamento_final.loc[df_orcamento_final['Vl_Orcado_Ano'].isna(),
                       'Vl_Orcado_Ano'] = 0
df_orcamento_final.loc[df_orcamento_final['Vl_Orcado_Atualizado'].isna(),
                       'Vl_Orcado_Atualizado'] = 0
df_orcamento_final.loc[df_orcamento_final['NIVEL_REGIONALIZAÇÃO'].isna(),
                       'NIVEL_REGIONALIZAÇÃO'] = 'Não detalhado'

df_orcamento_final

### Padronizando os nomes de subprefeituras

In [ ]:
subs_orcamento = (
    df_orcamento_final.loc[~df_orcamento_final['SUBPREFEITURA'].isna(), 'SUBPREFEITURA']
    .apply(unidecode)
    .unique()
    .tolist()
)

subs_orcamento.sort()

subs_orcamento

In [ ]:
subs_orcamento[:-6]

In [ ]:
len(subs_orcamento[:-6])

In [ ]:
subs_qlik_orcamento = subs_qlik.copy()
# As 3 subs abaixo não aparecem na lista de liquidação do orçamento
subs_qlik_orcamento.remove('ERMELINO MATARAZZO')
subs_qlik_orcamento.remove('PIRITUBA-JARAGUA')
subs_qlik_orcamento.remove('VILA MARIANA')
# Guainases e Vila Prudente aparecem em uma ordenação diferente, por isso serão
# removidas e adicionadas novamente ao final da lista
subs_qlik_orcamento.remove('GUAIANASES')
subs_qlik_orcamento.remove('VILA PRUDENTE')
subs_qlik_orcamento.append('GUAIANASES')
subs_qlik_orcamento.append('VILA PRUDENTE')

In [ ]:
mapper_orcamento = {
    so: sq
    for so, sq in zip(subs_orcamento, subs_qlik_orcamento)
}

mapper_orcamento

In [ ]:
df_orcamento_final.insert(
    7,
    'sub.NOME',
    df_orcamento_final.loc[:,'SUBPREFEITURA'].apply(lambda s: unidecode(s) if isinstance(s, str) else None).map(mapper_orcamento)
)

df_orcamento_final

In [ ]:
df_orcamento_final.loc[:, 'ano'] = 2024

# df_orcamento_final = df_orcamento_final.merge(df_subs_ano,
#                               how='left',
#                               on=['sub.NOME', 'ano'])

df_orcamento_final

### Adicionando as subprefeituras faltantes

In [ ]:
orcamento_subs_cols = ['CÓDIGO_ÓRGÃO', 'SIGLA_ÓRGÃO', 'DESCRIÇÃO_ÓRGÃO',
                       'CÓDIGO_PROGRAMA', 'DESCRIÇÃO_PROGRAMA',
                       'CÓDIGO_PROJ_ATIV', 'DESCRIÇÃO_PROJ_ATIV',
                       'CÓDIGO_VÍNCULO_PMSP', 'ano']

df_orcamento_subs = df_orcamento_final[orcamento_subs_cols].copy()
df_orcamento_subs = df_orcamento_subs.drop_duplicates().reset_index(drop=True)
df_orcamento_subs

In [ ]:
df_orcamento_subs['TIPO_REGIONALIZAÇÃO'] = 'Despesa Regionalizável'
df_orcamento_subs['NIVEL_REGIONALIZAÇÃO'] = 'Subprefeitura'

df_orcamento_subs

In [ ]:
df_orcamento_subs = (
    df_orcamento_subs
    .merge(pd.DataFrame(columns=['sub.NOME'], data=subs_qlik),
           how='cross')
)

df_orcamento_subs

In [ ]:
df_orcamento_completo = (
    df_orcamento_final
    .merge(df_orcamento_subs,
           how='outer',
           on=df_orcamento_subs.columns.tolist())
)

df_orcamento_completo

In [ ]:
df_orcamento_completo.loc[df_orcamento_completo['Vl_Liquidado'].isna(), 'Vl_Liquidado'] = 0
df_orcamento_completo.loc[df_orcamento_completo['Vl_Orcado_Ano'].isna(), 'Vl_Orcado_Ano'] = 0
df_orcamento_completo.loc[df_orcamento_completo['Vl_Orcado_Atualizado'].isna(), 'Vl_Orcado_Atualizado'] = 0

df_orcamento_completo

### Adicionando descrição das vinculações

In [ ]:
df_orcamento_completo = (
    df_orcamento_completo
    .merge(df_orcamento_original[['COD_VINC_REC_PMSP', 'TXT_VINC_PMSP']].drop_duplicates(),
            how='left',
            left_on='CÓDIGO_VÍNCULO_PMSP',
            right_on='COD_VINC_REC_PMSP')
    .drop(columns='COD_VINC_REC_PMSP')
)

df_orcamento_completo

### Adicionando a chave composta subprefeitura-ano

In [ ]:
df_orcamento_completo = df_orcamento_completo.merge(df_subs_ano,
                                                    how='left',
                                                    left_on=['sub.NOME', 'ano'],
                                                    right_on=['sub.NOME', 'ano'])

df_orcamento_completo

# Armazenamento

Finalmente, vamos exportar os dados em formato csv compatível com o Qlik e no padrão do excel para português do Brasil.

In [ ]:
base_path = path.join('data_output', 'urbanismo')

if not path.exists(base_path):
    makedirs(base_path)

for name, df in [('orcamento-habitacao', df_orcamento_final),
                 ('producao-his', df_his),
                 ('pdm-meta-12', df_meta_12),
                 ('emissoes-tpu', df_tpu),
                 ('subprefeitura-ano', df_subs_ano),
                 ('domicilios-favela', df_fav_sub_final),]:

    filepath = path.join(base_path, f'{name}.csv')

    df.to_csv(filepath,
              index=False,
              sep=';',
              decimal=',',
              encoding='latin1')